In [ ]:
from llama_cloud_services import LlamaCloudIndex
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.agent.workflow import FunctionAgent
import os
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Context


/media/aumoza/Strg_1/Freelance/personal/Chatbot/chatbot/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/media/aumoza/Strg_1/Freelance/personal/Chatbot/chatbot/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = Ollama(
    model="llama3.1:latest",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [4]:
def order(item:str) -> str:
    ''''Order items in the cart'''
    print(f"Ordered your {item}")

In [5]:
wishlist_items = []
cart = []

In [6]:
def addToCart(item)->str:
    """Useful for adding an item to the cart"""
    cart.append(item)
    return "Item added to the cart"

In [7]:
def wishlist(item)->str:
    """Useful for adding an item to the wishlist"""
    wishlist_items.append(item)
    return f"Added {item} to your wishlist."

In [8]:
agent = FunctionAgent(
    tools=[wishlist,addToCart],
    llm=Ollama(model="qwen3:0.6b",thinking=False),
    system_prompt="""You are a helpful assistant that can : 
    1. Add items to wishlist,
    2. Add an item to the cart.
    """,
)

In [9]:
query = "Add Modern Wooden Dining Table in cart"
# nodes = index.as_retriever().retrieve(query)
response = index.as_query_engine().query(query)

NameError: name 'index' is not defined

In [10]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = Ollama(
    model="llama3.2:latest",
    request_timeout=360.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [11]:
# Create a RAG tool using LlamaIndex
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(
    documents,
    # we can optionally override the embed_model here
    # embed_model=Settings.embed_model,
)
query_engine = index.as_query_engine(
    # we can optionally override the llm here
    # llm=Settings.llm,
)

In [ ]:
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    llm = model,
    chat_mode="context",
    memory=memory,
    system_prompt = (
    "You are a helpful and friendly shopping assistant chatbot. "
),
)

: 

In [80]:
response = chat_engine.chat("This one looks good : ErgoComfort")

In [81]:
response.response

"The ErgoComfort Office Chair is a popular choice among our customers. It's designed to provide excellent comfort and support for long hours at your desk.\n\nI've added the ErgoComfort Office Chair to your shopping cart. You can now check out and complete your order.\n\nHere's a summary of your updated cart:\n\n* Product: Modern Wooden Dining Table\n* Price: ₹18,999\n* Currency: INR\n\nAnd here's an update on your shopping cart:\n* Product: ErgoComfort Office Chair\n* Price: ₹7,999\n* Currency: INR\n\nYour total order now looks like this: ₹26,998\n\nWould you like to proceed with the payment or add any other products to your cart?"